In [0]:
pip install 'sqlalchemy < 2.0'

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 15.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
pip install 's3fs'

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Not uninstalling fsspec at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-854e2c8e-cb45-42d6-bac1-d12a127a2bae
    Can't uninstall 'fsspec'. No files were found to uninstall.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.28
    Not uninstalling botocore at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-854e2c8e-cb45-42d6-bac1-d12a127a2bae
    Can't uninstall 'botocore'. No files were found to uninstal

In [0]:
import pandas as pd
import pickle
import numpy as np
import subprocess
from sqlalchemy import create_engine
from datetime import timedelta
from sqlalchemy import Table, MetaData 
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs
import math
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit
import boto3
import pandas as pd
import numpy as np
import subprocess
import s3fs
import joblib
import datetime
import matplotlib.ticker as mtick
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructType, StructField, FloatType, StringType, BooleanType, ArrayType, IntegerType
from pyspark.sql import functions as F, Window
import pyspark.sql.types as T
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
#blocked out my AWS credentials for security
user = '****'
password = '****'
access_key = '****'
secret_access_key = '****'

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_access_key)

jdbcURL_staging = f"jdbc:postgresql://fw-rs-staging-us-east-1.cynyvmgqjdf4.us-east-1.redshift.amazonaws.com:5439/fwdb?user={user}&password={password}"

jdbcURL_qa = f"jdbc:postgresql://fw-rs-qa.cynyvmgqjdf4.us-east-1.redshift.amazonaws.com:5439/fwdb?user={user}&password={password}"

jdbcURL_prod = f"jdbc:postgresql://flywheel.cynyvmgqjdf4.us-east-1.redshift.amazonaws.com:5439/fwdb?user={user}&password={password}"

In [0]:
def get_wm_sales_brand(start_date, end_date, client_name, country):
  #country = country.upper()
  wm_sql_script = '''
WITH CTE AS (SELECT DISTINCT 
  B.name as business_unit, 
  lower(A.brand_segmentation) as brand, 
  A.walmart_campaign_id campaignid,
  nvl(A.item_segmentation, 'noSku') sku, 
  A.targeting_type,
  F.campaign_name, 
  E.code as country
FROM walmart.vw_walmart_segmentation A
JOIN ams.t_business_unit B on A.business_unit_id = B.id
JOIN ams.t_client_region C on B.client_country_id = C.client_country_id
JOIN ams.t_client_country D on C.client_country_id = D.id
JOIN ams.t_country E on D.country_id = E.id
JOIN walmart.dim_walmart_campaign F on A.walmart_campaign_id = F.id
WHERE b.name = '{client_name}'
AND lower(A.targeting_type) != 'auto'
AND E.code = '{country}'
AND F.campaign_name LIKE 'SP%'
  )
    SELECT
    B.country, 
    B.business_unit, 
    B.sku, 
    B.brand, 
    B.targeting_type,
    A.formatted_date as date, 
    cast(dateadd(d, cast(-date_part(dow, A.formatted_date) as int), A.formatted_date) as date) firstdayofweek,
    A.biddedkeyword, 
    A.searchedkeyword,
    SUM(A.advertisedskusales14days + A.otherskusales14days) sales, 
    SUM(A.adspend) spend, 
    A.campaignid
    FROM walmart.searched_keyword_dly A 
      JOIN CTE B ON A.campaignid = B.campaignid
    WHERE A.formatted_date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY A.formatted_date, B.country, B.business_unit, B.brand, B.sku, B.targeting_type, A.biddedkeyword, A.searchedkeyword, A.campaignid
    '''.format(start_date = start_date, end_date = end_date, client_name = client_name, country = country)
    
  wm_sales = spark.read \
          .format("com.databricks.spark.redshift") \
          .option("url", jdbcURL_prod) \
          .option("query", wm_sql_script) \
          .option("tempdir", "s3a://fw-advanced-analytics/Zac/tmp") \
          .option("forward_spark_s3_credentials", "true") \
          .load()

  
    
  return wm_sales

In [0]:
def get_median_daily_rank_per_keyword(start_date, end_date, country, kw_list_sql):
  from datetime import datetime, timedelta
  orig_start_date = start_date
  orig_end_date = end_date
  orig_start_date = start_date + ' 00:00:00.000000'
  orig_end_date = end_date + ' 23:59:59.000000'
  
  start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
  end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')

# Subtract one day from start_date and add one day to end_date
  new_start_date = start_date_obj - timedelta(days=1)
  new_end_date = end_date_obj + timedelta(days=1)

# Convert back to strings
  start_date = new_start_date.strftime('%Y-%m-%d')
  end_date = new_end_date.strftime('%Y-%m-%d')
  
  start_date = start_date + ' 00:00:00.000000'
  end_date = end_date + ' 23:59:59.000000'

    
  AWS_ACCESS_KEY="****"
  AWS_SECRET_KEY="******"
  AWS_SESSION_TOKEN="*****"
  AWS_REGION = 'us-east-1'
  SCHEMA_NAME = 'AwsDataCatalog'
  S3_STAGING_DIR = 's3://athena-dc-datalake-query-results/zac/tmp'

    # Initialize Athena client
  athena_client = boto3.client(
          "athena",
          aws_access_key_id=AWS_ACCESS_KEY,
          aws_secret_access_key=AWS_SECRET_KEY,
          region_name=AWS_REGION, 
          aws_session_token = AWS_SESSION_TOKEN
      )
    #countries = ('US', 'CA')
    
  query_string = """
WITH CTE AS (
    SELECT
        A.asin_sku as sku,
        A.keyword,
        (A.utctime AT TIME ZONE 'UTC' AT TIME ZONE 'America/Los_Angeles') as pdttime,
        A.absolute_rank as rank
    FROM "prd-silver".sov_data A
    WHERE
        A.country = '{country}'
        AND A.retailer = 'Walmart.com'
        AND A.utctime BETWEEN CAST('{start_date}' AS TIMESTAMP) AND CAST('{end_date}' AS TIMESTAMP)
        AND A.placement = 'Organic'
        AND A.keyword in ({kw_list_sql})
)
    SELECT 
        sku, 
        keyword,
        pdttime,
        dense_rank() over (partition by keyword, pdttime order by rank asc) as rank 
    FROM CTE
    WHERE pdttime BETWEEN CAST('{orig_start_date}' AS TIMESTAMP) AND CAST('{orig_end_date}' AS TIMESTAMP)

  """.format(start_date=start_date, end_date=end_date, country=country, orig_start_date = orig_start_date, orig_end_date = orig_end_date, kw_list_sql = kw_list_sql)
    

  # Run the query and get the results
  query_response = athena_client.start_query_execution(
      QueryString=query_string,
      QueryExecutionContext={"Database": SCHEMA_NAME},
      ResultConfiguration={
          "OutputLocation": "s3://datadrop-test/fw_fish/tmp/",
          "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
      }
  )

  query_execution_id = query_response["QueryExecutionId"]

  # Check query execution status
  query_execution = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
  query_execution_status = query_execution["QueryExecution"]["Status"]["State"]

  while query_execution_status in ["QUEUED", "RUNNING"]:
      query_execution = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
      query_execution_status = query_execution["QueryExecution"]["Status"]["State"]
          
      if query_execution_status == "FAILED":
          error_message = query_execution["QueryExecution"]["Status"]["StateChangeReason"]
          raise Exception(f"Athena query failed: {error_message}")

  # Initialize the next_token variable
  next_token = None

  # Initialize an empty list to store all rows
  all_rows = []

  # Retrieve the first set of results to get the header
  first_query_results = athena_client.get_query_results(QueryExecutionId=query_execution_id)
  header = [field["VarCharValue"] for field in first_query_results["ResultSet"]["Rows"][0]["Data"]]

  # Extract rows from the first set of results
  first_rows = [tuple(row["Data"][i]["VarCharValue"] for i in range(len(row["Data"]))) for row in first_query_results["ResultSet"]["Rows"][1:]]
  all_rows.extend(first_rows)

  # Set next_token if available
  next_token = first_query_results.get('NextToken', None)

  # Loop to handle pagination for subsequent pages
  while next_token:
      # Get the query results for the current page
      query_results = athena_client.get_query_results(QueryExecutionId=query_execution_id, NextToken=next_token)

      # Extract rows and extend the all_rows list
      rows = [tuple(row["Data"][i]["VarCharValue"] for i in range(len(row["Data"]))) for row in query_results["ResultSet"]["Rows"]]
      all_rows.extend(rows)

      # Update the next_token
      next_token = query_results.get('NextToken', None)

  # Create the Spark DataFrame
  sov = spark.createDataFrame(all_rows, header)

  # Persist the DataFrame
  sov.persist()


  sov = sov.withColumn('date', F.date_format(F.col('pdttime'), 'yyyy-MM-dd'))\
              .withColumn("dayofweek", F.dayofweek(F.col('pdttime')))\
              .withColumn("hour", F.hour(F.col('pdttime')))

  sov = sov.withColumn("dayofweek2", F.dayofweek(F.col("date")) - 1)

# Subtract the day of the week from the date to get the first day of the week (Sunday)
  sov = sov.withColumn("firstdayofweek", F.expr("date_sub(date, dayofweek2)"))
  sov = sov.drop("dayofweek2")      
      
  sov.createOrReplaceTempView('sov_tbl')
  n_scrapes_keyword = spark.sql('select keyword, date, count(distinct pdttime) as n_scrapes from sov_tbl group by keyword, date order by date')
  n_scrapes_keyword.persist()
      
  def hourly_weight_us():
          df = pd.read_csv('/dbfs/mnt/fw-advanced-analytics/Fish/organic_search_impact/by_hour_of_week_data/Hershey.csv')
          df = df.dropna(subset = ['event_hour_utc'])
          df['event_hour_utc'] = df['event_hour_utc'].astype(int)
          df['utc_time'] = df['event_date_utc'] + ' ' + df['event_hour_utc'].astype(str) + ':00:00'
          df['utc_time'] = pd.to_datetime(df['utc_time'], format = '%Y-%m-%d %H:%M:%S')
          df = spark.createDataFrame(df)
          df = df.withColumn('pdt_time', F.from_utc_timestamp(df['utc_time'], 'PST'))
          df = df.withColumn("dayofweek", F.dayofweek(F.col("pdt_time")))\
            .withColumn("hour", F.hour(F.col("pdt_time")))

          by_hour_of_week = df.groupby(['dayofweek', 'hour']).agg(F.sum('product_sales').alias('product_sales'), F.sum('clicks').alias('clicks'))
          by_hour_of_week = by_hour_of_week.withColumn('product_sales_by_clicks', by_hour_of_week.product_sales/by_hour_of_week.clicks)
          by_hour_of_week = by_hour_of_week.withColumn('weight', (by_hour_of_week.agg(F.max('product_sales_by_clicks')).collect()[0][0])/by_hour_of_week.product_sales_by_clicks)
          return by_hour_of_week.select(['dayofweek', 'hour', 'weight'])
      
      ####
      ## Multiplying Factor for Hershey: US
      ####
      
  by_hour_of_week = hourly_weight_us()
      
  def find_median(values_list):
      try:
          median = np.nanmedian(values_list) #get the median of values in a list in each row
          return round(float(median), 2)
      except Exception:
          return None       # if there is anything wrong with the given values

  median_finder = F.udf(find_median,T.FloatType())

  sov_weight = sov.join(by_hour_of_week, on = ['dayofweek', 'hour'], how = 'left')
  sov_weight = sov_weight.withColumn('rank_rescaled', sov_weight.rank * sov_weight.weight)

  median_rank = sov_weight.groupBy(['keyword', 'sku', 'date', 'firstdayofweek']).agg(F.collect_list("rank_rescaled").alias("rescaled_ranks"))
  median_rank = median_rank.withColumn("median_rank_rescaled", median_finder("rescaled_ranks"))

  median_rank = median_rank.withColumn('date',  F.to_timestamp(F.col('date'), 'yyyy-MM-dd'))

  median_rank_joined = median_rank.join(n_scrapes_keyword, on = ['keyword','date'], how = 'left')
  daily_observation = sov.groupBy(['sku', 'date', 'keyword']).count()
  daily_observation = daily_observation.withColumnRenamed('count', 'n_observations')
  median_rank_joined2 = median_rank_joined.join(daily_observation, on = ['sku', 'date', 'keyword'], how = 'inner')
  median_rank_joined2 = median_rank_joined2.withColumn('final_rank', F.sqrt(median_rank_joined2.n_scrapes/median_rank_joined2.n_observations) * median_rank_joined2.median_rank_rescaled)
      
  median_rank_joined2.createOrReplaceTempView('sov_tb2')
      
  final_daily_rank = spark.sql("""
                          select * 
                          from (
                          select *, 
                          rank() over (partition by date, keyword order by final_rank asc) as adjusted_rank
                          from sov_tb2 
                          )
                          order by date, keyword, adjusted_rank
                          """)
                          
  final_daily_rank = final_daily_rank.select(['sku','date', 'keyword', 'firstdayofweek', 'adjusted_rank'])
    #final_daily_rank = final_daily_rank.withColumn('date', F.date_format(F.col('date'), 'yyyy-MM-dd'))
  return final_daily_rank

In [0]:
def get_weekly_clickshare_brand(start_date, end_date, client_name,  country, department):
  country = country.upper()
  click_share_sql_script =  """
WITH CTE AS (
  SELECT DISTINCT 
  B.name as business_unit, 
  lower(A.brand_segmentation) as brand, 
  A.walmart_campaign_id campaignid,
  nvl(A.item_segmentation, 'noSku') sku, 
  A.targeting_type,
  F.campaign_name, 
  E.code as country
FROM walmart.vw_walmart_segmentation A
JOIN ams.t_business_unit B on A.business_unit_id = B.id
JOIN ams.t_client_region C on B.client_country_id = C.client_country_id
JOIN ams.t_client_country D on C.client_country_id = D.id
JOIN ams.t_country E on D.country_id = E.id
JOIN walmart.dim_walmart_campaign F on A.walmart_campaign_id = F.id
WHERE b.name = '{client_name}'
AND E.code = '{country}'
),
keywords_cte as (
SELECT 
  distinct A.searchedkeyword as keyword
FROM walmart.searched_keyword_dly A
JOIN CTE B ON A.campaignid = B.campaignid
WHERE A.formatted_date between '{start_date}' and '{end_date}'
)
select 
  search_term  keyword,
  start_date firstdayofweek,
  ROUND(2*(no_one_click_share + no_two_click_share + no_three_click_share), 0)/2.0 as top_three_click_share
FROM vc.fact_search_terms_weekly
WHERE search_term in (select keyword from keywords_cte)
AND start_date >= (select dateadd(d, cast(-date_part(dow,'{start_date}') as int), '{start_date}'))
AND start_date <= '{end_date}'
AND region = '{country}'
AND interval = 'Weekly'
AND department = '{department}'
             """.format(start_date = start_date, 
                        end_date = end_date, 
                        country = country,
                        client_name = client_name,
                        department = 'Amazon.com'
                        )
    
  click_share = spark.read \
          .format("com.databricks.spark.redshift") \
          .option("url", jdbcURL_prod) \
          .option("query", click_share_sql_script) \
          .option("tempdir", "s3a://fw-advanced-analytics/Zac/tmp") \
          .option("forward_spark_s3_credentials", "true") \
          .load()
  weights = pd.read_csv("/dbfs/mnt/fw-advanced-analytics/Fish/incrementality/clickshare_decay_100pct_95pct_90pct_first_page_exponential.csv")
  weights = weights[weights.first_page_clickshare == 100.0]
  weights = weights[['top3_clickshare', 'decay']]
  weights = spark.createDataFrame(weights)
  cond = [weights.top3_clickshare == click_share.top_three_click_share]
  joined = click_share.join(F.broadcast(weights), on = cond, how = 'inner')
  return joined.select('keyword', 'firstdayofweek', 'top_three_click_share', 'decay')

In [0]:
def get_conversion_brand(start_date, end_date, client_name, country):
  conversion_sql_script = """
WITH CTE AS (SELECT DISTINCT 
  B.name as business_unit, 
  lower(A.brand_segmentation) as brand, 
  A.walmart_campaign_id campaignid,
  nvl(A.item_segmentation, 'noSku') sku, 
  A.targeting_type,
  F.campaign_name, 
  E.code as country
FROM walmart.vw_walmart_segmentation A
JOIN ams.t_business_unit B on A.business_unit_id = B.id
JOIN ams.t_client_region C on B.client_country_id = C.client_country_id
JOIN ams.t_client_country D on C.client_country_id = D.id
JOIN ams.t_country E on D.country_id = E.id
JOIN walmart.dim_walmart_campaign F on A.walmart_campaign_id = F.id
WHERE b.name = '{client_name}'
AND lower(A.targeting_type) != 'auto'
AND E.code = '{country}'
AND F.campaign_name LIKE 'SP%'
  )
    SELECT 
    B.sku,
    A.formatted_date as date,
    cast(dateadd(d, cast(-date_part(dow, A.formatted_date) as int), A.formatted_date) as date) firstdayofweek,
    SUM(A.unitssold14d) units_sold,
    SUM(A.numadsclicks) clicks, 
    SUM(A.advertisedskusales14days + A.otherskusales14days) sales
    FROM walmart.searched_keyword_dly A
      JOIN CTE B ON A.campaignid = B.campaignid
    WHERE A.formatted_date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY date, B.sku
             """.format(client_name = client_name, 
                        country = country, 
                        start_date = start_date, 
                        end_date = end_date)
  df = spark.read \
          .format("com.databricks.spark.redshift") \
          .option("url", jdbcURL_prod) \
          .option("query", conversion_sql_script) \
          .option("tempdir", "s3a://fw-advanced-analytics/Zac/tmp") \
          .option("forward_spark_s3_credentials", "true") \
          .load()
  
  #df = df.groupBy("client_name", "country", "business_unit", "sku", "brand", "firstdayofweek", "date", "keyword").agg(sum("units_sold").alias("units_sold"), sum("clicks").alias("clicks"))
  df = df.filter(df['clicks'] > 0)

  df.createOrReplaceTempView('tmp')
  df = spark.sql("""
                 select sku, 
                        firstdayofweek, 
                        date,
                        cast(units_sold as float)/cast(clicks as float) as conversion,
                        cast(sales as float)/cast(units_sold as float) as asp
                 from tmp
                 """)

  return df

In [0]:
def get_incremental_sales(busisness_unit): 
  from pyspark.sql.functions import when

  m_start = '2023-12-31'
  m_end = '2024-04-09'
  #bu = "'Danone','Hersheys - US','Mattel - US', 'McCormick', 'Nestle - Purina', 'Nestle Health Science', 'Samsung TV', 'WhyteSpyder Bayer US', 'Clorox', 'Haleon-Consumer-Healthcare US', 'Revlon'"
  bu = business_unit
  wm_sales_all = get_wm_sales_brand(m_start, m_end, bu ,'US')
  wm_sales_all.persist()

  wm_sales_all = wm_sales_all.withColumn(
      "brand",
      F.when(
          (F.col("brand").isNull()) & (F.col("business_unit") == "Samsung TV"),
          F.lit("samsung")
      ).otherwise(F.col("brand"))
  )
  wm_sales_all = wm_sales_all.withColumn(
      "brand",
      F.when(
          (F.col("brand").isNull()) & (F.col("business_unit") == "WhyteSpyder Bayer US"),
          F.lit("bayer")
      ).otherwise(F.col("brand"))
  )

  weekly_clickshare = get_weekly_clickshare_brand(m_start, m_end, bu, 'US', 'Amazon.com')
  weekly_clickshare.persist()

  #Prioritize searchedkeyword, use bidded if searched does not exist in clickshare table
  wm_sales_all = wm_sales_all.alias("wm")
  clickshare = weekly_clickshare.alias("cs")
  # Perform a left join between wm_sales_all and clickshare
  joined_df = wm_sales_all.join(
      clickshare,
      (F.col("wm.firstdayofweek") == F.col("cs.firstdayofweek")) & 
      (F.col("wm.searchedkeyword") == F.col("cs.keyword")),
      'left'
  )
  # Create a new column 'keyword' in wm_sales_all
  # Use 'searchedkeyword' if there's a match, otherwise use 'biddedkeyword'
  wm_sales_all = joined_df.withColumn(
      "keyword",
      F.when(F.col("cs.keyword").isNotNull(), F.col("wm.searchedkeyword"))
      .otherwise(F.col("wm.biddedkeyword"))
  )
  # Drop the 'searchedkeyword' and 'biddedkeyword' columns
  wm_sales_all = wm_sales_all.drop("searchedkeyword", "biddedkeyword", "top_three_click_share", "decay")
  # Optionally, drop the extra columns from clickshare if they were joined
  #wm_sales_all = wm_sales_all.drop(*[column for column in clickshare.columns if column != 'keyword' o column != 'firstdayofweek'])

  wm_sales = wm_sales_all.join(weekly_clickshare,  on = ['keyword', 'firstdayofweek'], how = 'leftsemi') # if keyword does not have sfr, exclude its sales
  wm_sales.persist()

  unique_kws = wm_sales.select("keyword").distinct()
  unique_kws_list = [row.keyword for row in unique_kws.collect()]
  unique_kws = tuple(unique_kws_list)
  unique_kws = tuple(brand.replace("'", "''") for brand in unique_kws)    
  kw_list_sql = ', '.join(f"'{kw}'" for kw in unique_kws)
  daily_rank = get_median_daily_rank_per_keyword(m_start, m_end, 'US', kw_list_sql)
  daily_rank.persist()
  #Add brands to daily_rank
  sales_brands = wm_sales_all.createOrReplaceTempView('brands_tbl')
  sales_brands = spark.sql("""
    SELECT DISTINCT sku, brand
    FROM brands_tbl
  """)
  from pyspark.sql.functions import col
  sales_brands = sales_brands.filter((col("sku") != 'SB/SBV') & col("sku").isNotNull() & (col("sku") != 'noSku')) #Do not include sales not associated with an SKU
  daily_rank = daily_rank.join(sales_brands, on = ["sku"], how = "left")

  conversion = get_conversion_brand(m_start, m_end, bu, 'US')
  conversion.persist()

  daily_rank_conversion = daily_rank.join(conversion, on = ['sku', 'date'], how = 'left') 
  daily_rank_conversion.persist()

  rank_clickshare = daily_rank_conversion.join(weekly_clickshare, on = ['keyword', 'firstdayofweek'], how = 'left')
  rank_clickshare = rank_clickshare.withColumn('distribution', F.pow(rank_clickshare.decay, rank_clickshare.adjusted_rank))
  rank_clickshare.persist() #Wegihts steepness of clickshare curve into rank

  rank_clickshare_copy = rank_clickshare.toDF(*[col + '2' for col in rank_clickshare.columns])
  cond = [rank_clickshare.brand == rank_clickshare_copy.brand2, 
        rank_clickshare.keyword == rank_clickshare_copy.keyword2,
        rank_clickshare.date == rank_clickshare_copy.date2]
  rank_clickshare_joined = rank_clickshare.join(rank_clickshare_copy, on = cond, how = 'outer')
  rank_clickshare_joined.persist()

  rank_clickshare_joined.createOrReplaceTempView('rank_clickshare_tbl')
  same_brand = spark.sql("""
    select keyword, sku, date, brand,  sum(distribution2 * weight) as same_brand_distribution
        from (
      select *, 
      case 
      when (conversion2 is not null and conversion is not null and asp is not null and asp2 is not null) and  ((asp2 * conversion2) < (asp * conversion)) then 0.8 
      when ((conversion2 is null or conversion is null) and (asp is not null and asp2 is not null)) and  (asp2  > asp) then 0.8 
      else 1.0 
      end as weight
      from rank_clickshare_tbl 
      where sku != sku2 and distribution <= distribution2
        )
      group by keyword, sku, date, brand
      """)
  same_brand.persist()

  rank_clickshare2 = rank_clickshare.join(same_brand, on = ['keyword', 'sku', 'date', 'brand'], how = 'left')
  rank_clickshare2 = rank_clickshare2.withColumn(
      'same_brand_distribution_unscaled', 
      F.sum('distribution').over(
        Window.partitionBy(['brand', 'keyword', 'date'])
            .orderBy(F.desc('distribution'))
            .rowsBetween(Window.unboundedPreceding, -1)
      )
      ).withColumn(
      'different_brand_distribution', 
      F.sum('distribution').over(
        Window.partitionBy(['keyword', 'date']).orderBy(F.desc('distribution'))
            .rowsBetween(Window.unboundedPreceding, -1)
      ) - F.coalesce(F.col('same_brand_distribution_unscaled'), F.lit(0))
      )
  rank_clickshare2.persist()
  rank_clickshare2 = rank_clickshare2.withColumn('prob_different_brand', F.col('different_brand_distribution')/(F.col('different_brand_distribution') + F.col   ('same_brand_distribution'))).fillna(value = 1, subset = ['prob_different_brand'])
    
  rank_clickshare2.createOrReplaceTempView('rank_clickshare_tbl2')
  rank_clickshare3 = spark.sql("""
      select 
          keyword, 
          sku, 
          brand,
          date, 
          adjusted_rank,
          top_three_click_share, 
          decay, 
          distribution,
          scaled_dist,
          percent_total,
          prob_different_brand,
          pag1_different_brand_distribution
      from (
        select *,  (1.0 + ((distribution - min_d) * 9) / nullif((max_d - min_d), 0))/10.0   as scaled_dist,
          nvl(max_different_brand_distribution/(max_different_brand_distribution + max_same_brand_distribution_unscaled), 1) as pag1_different_brand_distribution
        from (
        select *,  
          max(same_brand_distribution_unscaled) over (PARTITION BY date, keyword, brand) as max_same_brand_distribution_unscaled,
          max(different_brand_distribution) over (PARTITION BY date, keyword, brand) as max_different_brand_distribution,
          max(distribution) over (PARTITION BY date, keyword) as max_d,
          min(distribution) over (PARTITION BY date, keyword) as min_d,
          distribution /nullif((sum(distribution) over (PARTITION BY date, keyword)), 0) as percent_total
        from rank_clickshare_tbl2
        )
        )
        order by percent_total desc nulls last
      """)
  rank_clickshare3.persist()
  wm_sales_scrapped_keywords = wm_sales.join(rank_clickshare3, on = ['keyword', 'date'], how = 'leftsemi')
  joined_df = wm_sales_scrapped_keywords.join(rank_clickshare3, on = ['keyword', 'date', 'sku', 'brand'], how = 'left')
  rank_clickshare3_copy = rank_clickshare3.toDF(*[col + '2' for col in rank_clickshare3.columns])\
                                          .select('date2', 'brand2', 'keyword2', 'pag1_different_brand_distribution2').distinct()
  rank_clickshare3_copy.persist()
  cond = [joined_df.keyword == rank_clickshare3_copy.keyword2,
              joined_df.date == rank_clickshare3_copy.date2,
              joined_df.brand == rank_clickshare3_copy.brand2]
  joined_df2 = joined_df.join(rank_clickshare3_copy, on = cond, how = 'left')
  joined_df2.persist()

  joined_df2.createOrReplaceTempView('rank_clickshare_tbl3')
  incremental_sales = spark.sql("""
      select 
          keyword, 
          sku, 
          brand,
          business_unit,
          targeting_type,
          date, 
          adjusted_rank,
            top_three_click_share, 
            decay, 
            distribution,
            scaled_dist,
            percent_total,
            prob_different_brand,
            nvl(pag1_different_brand_distribution,1) as pag1_different_brand_distribution,
            nvl(pag1_different_brand_distribution2,1) as pag1_different_brand_distribution2,
          spend,
          sales,
          case when adjusted_rank <= 60 then sales * (1 - scaled_dist) * prob_different_brand
            else sales * nvl(nvl(prob_different_brand, pag1_different_brand_distribution), 
                          nvl(pag1_different_brand_distribution2, 1)) end as incremental_sales,
          case when adjusted_rank <= 60 then sales * (1 - scaled_dist) else sales end incremental_sales_older
            
      from  rank_clickshare_tbl3
        order by incremental_sales desc
      """)
  return incremental_sales

In [0]:
from pyspark.sql.functions import when

m_start = '2023-08-01'
m_end = '2024-02-29'
#bu = "'Danone','Hersheys - US','Mattel - US', 'McCormick', 'Nestle - Purina', 'Nestle Health Science', 'Samsung TV', 'WhyteSpyder Bayer US', 'Clorox', 'Haleon-Consumer-Healthcare US', 'Revlon'"
bu = 'SC Johnson'
wm_sales_all = get_wm_sales_brand(m_start, m_end, bu ,'US')
wm_sales_all.persist()

wm_sales_all = wm_sales_all.withColumn(
    "brand",
    F.when(
        (F.col("brand").isNull()) & (F.col("business_unit") == "Samsung TV"),
        F.lit("samsung")
    ).otherwise(F.col("brand"))
)
wm_sales_all = wm_sales_all.withColumn(
    "brand",
    F.when(
        (F.col("brand").isNull()) & (F.col("business_unit") == "WhyteSpyder Bayer US"),
        F.lit("bayer")
    ).otherwise(F.col("brand"))
)

weekly_clickshare = get_weekly_clickshare_brand(m_start, m_end, bu, 'US', 'Amazon.com')
weekly_clickshare.persist()

#Prioritize searchedkeyword, use bidded if searched does not exist in clickshare table
wm_sales_all = wm_sales_all.alias("wm")
clickshare = weekly_clickshare.alias("cs")
# Perform a left join between wm_sales_all and clickshare
joined_df = wm_sales_all.join(
    clickshare,
    (F.col("wm.firstdayofweek") == F.col("cs.firstdayofweek")) & 
    (F.col("wm.searchedkeyword") == F.col("cs.keyword")),
    'left'
)
# Create a new column 'keyword' in wm_sales_all
# Use 'searchedkeyword' if there's a match, otherwise use 'biddedkeyword'
wm_sales_all = joined_df.withColumn(
    "keyword",
    F.when(F.col("cs.keyword").isNotNull(), F.col("wm.searchedkeyword"))
     .otherwise(F.col("wm.biddedkeyword"))
)
# Drop the 'searchedkeyword' and 'biddedkeyword' columns
wm_sales_all = wm_sales_all.drop("searchedkeyword", "biddedkeyword", "top_three_click_share", "decay")
# Optionally, drop the extra columns from clickshare if they were joined
#wm_sales_all = wm_sales_all.drop(*[column for column in clickshare.columns if column != 'keyword' o column != 'firstdayofweek'])

wm_sales = wm_sales_all.join(weekly_clickshare,  on = ['keyword', 'firstdayofweek'], how = 'leftsemi') # if keyword does not have sfr, exclude its sales
wm_sales.persist()

unique_kws = wm_sales.select("keyword").distinct()
unique_kws_list = [row.keyword for row in unique_kws.collect()]
unique_kws = tuple(unique_kws_list)
unique_kws = tuple(brand.replace("'", "''") for brand in unique_kws)    
kw_list_sql = ', '.join(f"'{kw}'" for kw in unique_kws)
daily_rank = get_median_daily_rank_per_keyword(m_start, m_end, 'US', kw_list_sql)
daily_rank.persist()
#Add brands to daily_rank
sales_brands = wm_sales_all.createOrReplaceTempView('brands_tbl')
sales_brands = spark.sql("""
  SELECT DISTINCT sku, brand
  FROM brands_tbl
""")
from pyspark.sql.functions import col
sales_brands = sales_brands.filter((col("sku") != 'SB/SBV') & col("sku").isNotNull() & (col("sku") != 'noSku')) #Do not include sales not associated with an SKU
daily_rank = daily_rank.join(sales_brands, on = ["sku"], how = "left")

conversion = get_conversion_brand(m_start, m_end, bu, 'US')
conversion.persist()

daily_rank_conversion = daily_rank.join(conversion, on = ['sku', 'date'], how = 'left') 
daily_rank_conversion.persist()

rank_clickshare = daily_rank_conversion.join(weekly_clickshare, on = ['keyword', 'firstdayofweek'], how = 'left')
rank_clickshare = rank_clickshare.withColumn('distribution', F.pow(rank_clickshare.decay, rank_clickshare.adjusted_rank))
rank_clickshare.persist() #Wegihts steepness of clickshare curve into rank

rank_clickshare_copy = rank_clickshare.toDF(*[col + '2' for col in rank_clickshare.columns])
cond = [rank_clickshare.brand == rank_clickshare_copy.brand2, 
			rank_clickshare.keyword == rank_clickshare_copy.keyword2,
			rank_clickshare.date == rank_clickshare_copy.date2]
rank_clickshare_joined = rank_clickshare.join(rank_clickshare_copy, on = cond, how = 'outer')
rank_clickshare_joined.persist()

rank_clickshare_joined.createOrReplaceTempView('rank_clickshare_tbl')
same_brand = spark.sql("""
	select keyword, sku, date, brand,  sum(distribution2 * weight) as same_brand_distribution
		   from (
		select *, 
    case 
    when (conversion2 is not null and conversion is not null and asp is not null and asp2 is not null) and  ((asp2 * conversion2) < (asp * conversion)) then 0.8 
		when ((conversion2 is null or conversion is null) and (asp is not null and asp2 is not null)) and  (asp2  > asp) then 0.8 
     else 1.0 
     end as weight
		from rank_clickshare_tbl 
		where sku != sku2 and distribution <= distribution2
			)
		 group by keyword, sku, date, brand
		 """)
same_brand.persist()

rank_clickshare2 = rank_clickshare.join(same_brand, on = ['keyword', 'sku', 'date', 'brand'], how = 'left')
rank_clickshare2 = rank_clickshare2.withColumn(
		'same_brand_distribution_unscaled', 
		F.sum('distribution').over(
			Window.partitionBy(['brand', 'keyword', 'date'])
				  .orderBy(F.desc('distribution'))
				  .rowsBetween(Window.unboundedPreceding, -1)
		)
		).withColumn(
		'different_brand_distribution', 
		F.sum('distribution').over(
			Window.partitionBy(['keyword', 'date']).orderBy(F.desc('distribution'))
				  .rowsBetween(Window.unboundedPreceding, -1)
		) - F.coalesce(F.col('same_brand_distribution_unscaled'), F.lit(0))
		)
rank_clickshare2.persist()
rank_clickshare2 = rank_clickshare2.withColumn('prob_different_brand', F.col('different_brand_distribution')/(F.col('different_brand_distribution') + F.col   ('same_brand_distribution'))).fillna(value = 1, subset = ['prob_different_brand'])
    
rank_clickshare2.createOrReplaceTempView('rank_clickshare_tbl2')
rank_clickshare3 = spark.sql("""
		select 
				keyword, 
				sku, 
				brand,
				date, 
				adjusted_rank,
				top_three_click_share, 
				decay, 
				distribution,
				scaled_dist,
				percent_total,
				prob_different_brand,
				pag1_different_brand_distribution
		from (
		  select *,  (1.0 + ((distribution - min_d) * 9) / nullif((max_d - min_d), 0))/10.0   as scaled_dist,
		    nvl(max_different_brand_distribution/(max_different_brand_distribution + max_same_brand_distribution_unscaled), 1) as pag1_different_brand_distribution
		  from (
			select *,  
			   max(same_brand_distribution_unscaled) over (PARTITION BY date, keyword, brand) as max_same_brand_distribution_unscaled,
			   max(different_brand_distribution) over (PARTITION BY date, keyword, brand) as max_different_brand_distribution,
			   max(distribution) over (PARTITION BY date, keyword) as max_d,
			   min(distribution) over (PARTITION BY date, keyword) as min_d,
			  distribution /nullif((sum(distribution) over (PARTITION BY date, keyword)), 0) as percent_total
		   from rank_clickshare_tbl2
			 )
		   )
		  order by percent_total desc nulls last
		""")
rank_clickshare3.persist()
wm_sales_scrapped_keywords = wm_sales.join(rank_clickshare3, on = ['keyword', 'date'], how = 'leftsemi')
joined_df = wm_sales_scrapped_keywords.join(rank_clickshare3, on = ['keyword', 'date', 'sku', 'brand'], how = 'left')
rank_clickshare3_copy = rank_clickshare3.toDF(*[col + '2' for col in rank_clickshare3.columns])\
                                        .select('date2', 'brand2', 'keyword2', 'pag1_different_brand_distribution2').distinct()
rank_clickshare3_copy.persist()
cond = [joined_df.keyword == rank_clickshare3_copy.keyword2,
            joined_df.date == rank_clickshare3_copy.date2,
            joined_df.brand == rank_clickshare3_copy.brand2]
joined_df2 = joined_df.join(rank_clickshare3_copy, on = cond, how = 'left')
joined_df2.persist()

joined_df2.createOrReplaceTempView('rank_clickshare_tbl3')
incremental_sales = spark.sql("""
		select 
				keyword, 
				sku, 
				brand,
				business_unit,
				targeting_type,
				date, 
				adjusted_rank,
			    top_three_click_share, 
			    decay, 
			    distribution,
			    scaled_dist,
			    percent_total,
			    prob_different_brand,
			    nvl(pag1_different_brand_distribution,1) as pag1_different_brand_distribution,
			    nvl(pag1_different_brand_distribution2,1) as pag1_different_brand_distribution2,
				spend,
				sales,
				case when adjusted_rank <= 60 then sales * (1 - scaled_dist) * prob_different_brand
				  else sales * nvl(nvl(prob_different_brand, pag1_different_brand_distribution), 
				                nvl(pag1_different_brand_distribution2, 1)) end as incremental_sales,
				case when adjusted_rank <= 60 then sales * (1 - scaled_dist) else sales end incremental_sales_older
				  
		from  rank_clickshare_tbl3
		  order by incremental_sales desc
		""")
incremental_sales.persist()

DataFrame[keyword: string, sku: string, brand: string, business_unit: string, targeting_type: string, date: date, adjusted_rank: int, top_three_click_share: decimal(16,4), decay: double, distribution: double, scaled_dist: double, percent_total: double, prob_different_brand: double, pag1_different_brand_distribution: double, pag1_different_brand_distribution2: double, spend: decimal(38,6), sales: decimal(38,6), incremental_sales: double, incremental_sales_older: double]

In [0]:
from pyspark.sql import functions as F

# Add 'year_month' column to 'incremental_sales' DataFrame
incremental_sales = incremental_sales.withColumn('year_month', F.date_format(F.col('date'), 'yyyy-MM'))

# Get the distinct combinations of year-month and business units
combo_list = incremental_sales.select('year_month', 'business_unit').distinct().collect()

# Iterate over each combination of year-month and business unit
for row in combo_list:
    year_month = row.year_month
    business_unit = row.business_unit

    # Filter for the specific year-month and business unit
    filtered_df = incremental_sales.filter(
        (F.col('year_month') == year_month) & 
        (F.col('business_unit') == business_unit)
    )

    # Convert to Pandas DataFrame (ensure this is feasible size-wise)
    pandas_df = filtered_df.toPandas()

    # Define the file path, incorporating the business_unit dynamically
    file_path = f"/dbfs/mnt/fw-advanced-analytics/Zac/Walmart Incrementality Final Results/{business_unit}/{business_unit} {year_month}.csv"

    # Save to CSV
    pandas_df.to_csv(file_path, index=False)


/databricks/spark/python/pyspark/sql/pandas/utils.py:105: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [top_three_click_share, spend, sales] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
  warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:105: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [top_three_click_share, spend, sales] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
  warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:105: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [top_three_click_share, spend, sales] If those columns are not necessary, you may consider dropping them or converting to primitive types before the con